In [1]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions,SemanticRolesOptions,RelationsOptions
import codecs
import networkx as nx
import operator
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-11-16',
    iam_apikey='-2GVZ0naktrQbO23BmCMrRhIuT3rgjGOs1bHTdzJ-h3A',
    url='https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
)

In [3]:
def get_centralities(text):
    response = natural_language_understanding.analyze(
                    text=text,
                    features=Features(entities=EntitiesOptions(),
                                      keywords=KeywordsOptions(sentiment = True,emotion = True),
                                      semantic_roles=SemanticRolesOptions(keywords=True,entities=True),
                                      relations=RelationsOptions()),
                    language='en').get_result()
    key_sent = {}
    for words in response['keywords']:
        key_sent[words['text']] = words['sentiment']['score']
    key_emotion = {}
    for words in response['keywords']:
        key_emotion[words['text']] = words['emotion']
    keylist_rel = {}
    for key in response['keywords']:
        key_ent = key['text']
        rel = key['relevance']
        keylist_rel[key_ent] = rel
    entities_rel = {}
    G = nx.Graph()
    sub = None
    obj = None
    
    for role in response["semantic_roles"]:
        subject = role['subject']['text'].lower()

        max_rel = 0
        max_ent = []
        if('keywords' in role['subject'].keys()):
            for ent in role['subject']['keywords']:
                ent_text = ent['text'].lower()

                for dict_key in keylist_rel.keys():
                    if(ent_text in dict_key.lower()):
                            ent_rel =  keylist_rel[dict_key]

                            if(ent_rel>max_rel):
                                max_rel = ent_rel 
                                max_ent = ent
                if(max_rel!=0 and max_ent):
                    sub = max_ent['text']
                    G.add_node(sub)

        verb = role['action']['verb']['text'].lower()

        max_rel = 0
        max_ent = []
        if('object' in role.keys()):
            object = role['object']['text'].lower()
            if('keywords' in role['object'].keys()):
                for obj in role['object']['keywords']:
                    obj_text = obj['text'].lower()

                    for dict_key in keylist_rel.keys():
                        if(obj_text in dict_key.lower()):
                            obj_rel =  keylist_rel[dict_key]

                            if(obj_rel>max_rel):
                                max_rel = obj_rel 
                                max_ent = obj
                    if(max_rel!=0 and max_ent):
                        obj = max_ent['text']
                        G.add_node(obj)
                        if(sub and obj and sub!=obj):
                            G.add_edge(sub,obj)
    return nx.closeness_centrality(G), key_sent, key_emotion

In [4]:
def sort_dict(my_dict):
    return sorted(my_dict.items(), key=operator.itemgetter(1),reverse=True)

In [5]:
def get_match(mylist,list):
    maxval = 0
    r_ent_rel = [None,None]
    for tup in list:
        count = 0
        otherlist = tup[0].split(" ")
        for myword in mylist:
            for otherword in otherlist:
                if lemmatizer.lemmatize(myword.lower()) == lemmatizer.lemmatize(otherword.lower()):
                    count+=1
        if count > maxval:
            maxval = count
            r_ent_rel = [tup[0],tup[1]]
    return r_ent_rel

In [6]:
def does_contain(str1,str2):
    count = 0
    str1 = str1.split(" ")
    str2 = str2.split(" ")
    for word1 in str1:
        for word2 in str2:
            if(word1.lower() == word2.lower()):
                count+=1
    return count

In [7]:
def get_sent(str,dict):
    if str in dict :
        return dict[str]
    maxval = 0
    rval = None
    str = str.split(" ")
    for keys in dict:
        count = 0
        for mywords in str:
            for otherwords in keys.split(" "):
                if(mywords.lower() == otherwords.lower()):
                    count+=1
        if count > maxval:
            maxval = count
            rval = dict[keys]
    return rval   